In [1]:
import torch as th 
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

from model import Generator, Discriminator, gradient_penalty
from utils import PolyGraphDataset, Transformed_PolyGraphDataset, CATEGORY_DICT

import numpy as np

In [2]:
""" 
Hyperparameters
"""

MAX_POLYGONS = 30

# Optimizer params
g_lr = 0.001 
d_lr = 0.001
b1 = 0.5 
b2 = 0.999  

# WGAN params
N_critic = 5            # nr of times to train discriminator more
lambda_gp = 10          # gradient penalty hyperpraram

# Training params
MAX_EPOCHS = 500
BATCH_SIZE = 16

# Network parameters
NOISE_SIZE = 128
HIDDEN_GENERATOR = [128, 128, 128]
OUTPUT_GENERATOR = MAX_POLYGONS * 2             # we want to output at most this many polygons per node (note [x1...y1...] format)

HIDDEN_DISCRIMINATOR = [128, 128, 128]

In [3]:
""" 
Model Definitions
"""
generator = Generator(input_dim=NOISE_SIZE + len(CATEGORY_DICT), 
                      output_dim=OUTPUT_GENERATOR, 
                      hidden_dims=HIDDEN_GENERATOR)

discriminator = Discriminator(input_dim=OUTPUT_GENERATOR, 
                              hidden_dims=HIDDEN_DISCRIMINATOR)

print(generator.module_list)

ModuleList(
  (0): TAGConv(141, 128, K=3)
  (1): TAGConv(128, 128, K=3)
  (2): TAGConv(128, 60, K=3)
)


In [4]:
# Optimizers
optimizer_G = th.optim.Adam(generator.parameters(), lr=g_lr, betas=(b1, b2)) 
optimizer_D = th.optim.Adam(discriminator.parameters(), lr=d_lr, betas=(b1, b2))

In [5]:
# Main training loop
def train(generator, discriminator, optimizer_g, optimizer_d, data_loader):
    for epoch in range(MAX_EPOCHS):
        # real == batch (confusing naming I know...)
        for real in data_loader:
            # real = batch of Data() ex. [Data(), Data(), ...] is 1 batch
            # Create new data object with noise and same edge_index 
            print("real: ", real)   # should be a Batch() item

            for i in range(N_critic):
                # Input noise_data into generator
                global fake 
                
                noise = th.randn((len(real.category), NOISE_SIZE))
                # print("noise: ", noise.shape)
                fake = generator(real, noise)     

                # fake.shape = (batch_size * nodes, output_features = 60)
                # We must turn this into appropriate (batch) input for the discriminator
                fake = Batch(geometry=fake, edge_index=real.edge_index, batch=real.batch)
                
                # print("fake: ", fake.geometry.shape)
                # print("real: ", real.geometry.shape)

                discriminator_fake = discriminator(fake)    # discriminator scores for fakes
                discriminator_real = discriminator(real)    # discriminator scores for reals
                
                gp = gradient_penalty(discriminator, real, fake)

                # Discriminator loss and train
                loss_discriminator = -(th.mean(discriminator_real) - th.mean(discriminator_fake)) + lambda_gp * gp
                discriminator.zero_grad() 
                loss_discriminator.backward() 
                optimizer_d.step()

                return 0 

            # Generator loss and train
            output = discriminator(fake).reshape(-1)        # discriminator scores for fake
            loss_generator = -th.mean(output)               # loss for genereator = the discriminators' judgement
                                                            # higher score = better
            generator.zero_grad()
            loss_generator.backward()
            optimizer_g.step()
    
        # TODO: Evaluation and logging code??

Testing main training loop

Doesnt work yet due to dataloader iteration being broken

In [6]:
path = r'C:\School\DELFT\Graph_ML_project\data\swiss-dwellings-v3.0.0'
dataset = Transformed_PolyGraphDataset(path)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
n_batches = len(dataloader)
print(n_batches)

# train(generator, discriminator, optimizer_G, optimizer_D, dataloader)

587


Testing model for batches of input and gradient penalty for batches

In [7]:
from model import gradient_penalty

print(dataset[0])
test_list = [dataset[0] for _ in range(32)]
test_batch = Batch.from_data_list(test_list)
print(test_batch)

real = test_batch

noise = th.randn((len(real.category), NOISE_SIZE))
print("noise: ", noise.shape)
fake = generator(real, noise)     

# fake.shape = (batch_size * nodes, output_features = 60)
# We must turn this into appropriate (batch) input for the discriminator
fake = Batch(geometry=fake, edge_index=real.edge_index, batch=real.batch)

print("fake: ", fake.geometry.shape)
print("real: ", real.geometry.shape)

discriminator_fake = discriminator(fake)    # discriminator scores for fakes
discriminator_real = discriminator(real)    # discriminator scores for reals

print("fake: ", fake)
print("real: ", real)
print("discriminator score fake: ", discriminator_fake.shape)
print("discriminator score real: ", discriminator_real.shape)

gp = gradient_penalty(discriminator, real, fake)
print(gp)

Data(edge_index=[2, 72], geometry=[42, 60], category=[42, 13], door_geometry=[72], num_nodes=42)


DataBatch(edge_index=[2, 2304], geometry=[1344, 60], category=[1344, 13], door_geometry=[32], num_nodes=1344, batch=[1344], ptr=[33])
noise:  torch.Size([1344, 128])
fake:  torch.Size([1344, 60])
real:  torch.Size([1344, 60])
fake:  DataBatch(edge_index=[2, 2304], geometry=[1344, 60], batch=[1344])
real:  DataBatch(edge_index=[2, 2304], geometry=[1344, 60], category=[1344, 13], door_geometry=[32], num_nodes=1344, batch=[1344], ptr=[33])
discriminator score fake:  torch.Size([32, 1])
discriminator score real:  torch.Size([32, 1])
tensor(0.5840, grad_fn=<MeanBackward0>)


In [8]:
tens = th.ones(100)
print(th.mean(tens))

tensor(1.)


Testing dataloader iteration

In [9]:
from tqdm.auto import tqdm

""" 
Code below produces "AttributeError: 'list' object has no attribute 'size'"
"""
dataiter = iter(dataloader)

for _ in tqdm(range(n_batches)):
    batch = next(dataiter)
    print(batch)


c:\Users\haora\anaconda3\envs\graph_ml_course\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/587 [00:00<?, ?it/s]

  0%|          | 0/587 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'size'